In [1]:
# load libraies

%run py_libraries.py

/Users/4476224/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.13.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# loading utility files

from utility.sv_fig import savefig
from utility.make_cm import make_confusion_matrix
from utility.get_g_result import get_gamma_results
from utility.plt_result import plot_results

In [3]:
def savefig(filename, crop = True):
    plt.savefig('{}.pdf'.format(filename))

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

# load data
X_train = pd.read_csv('data/norm_X_train_NCa_Ca.csv')
X_test = pd.read_csv('data/norm_X_test_NCa_Ca.csv')
y_train = pd.read_csv('data/y_train_NCa_Ca.csv')
y_test = pd.read_csv('data/y_test_NCa_Ca.csv')


print(X_test.shape)

(40, 37)


In [5]:
# Print the column names as a list
print(X_test.columns.tolist())

['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 'Leptin', 'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 'Laminin', 'HbA1c', 'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG']


In [6]:
Xy_NCa_Ca = pd.concat([X_test, y_test], axis=1)

In [7]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from scipy import stats

col_to_scale = ['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 'Leptin', 
                'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 'Laminin', 'HbA1c', 
                'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG']

def midas_impl(missing, col_to_scale, n_epoch):
    missing_cp = missing.copy() # make copy of missing df
    
    # scaler = MinMaxScaler().fit(missing_cp[col_to_scale]) # scale data
    scaler = StandardScaler().fit(missing_cp[col_to_scale]) # scale data
    missing_sc = scaler.transform(missing_cp[col_to_scale]) # this is an array, convert to df

    missing_df = pd.DataFrame({'ENA.78':missing_sc[:,0],
                               'IFN.y':missing_sc[:,1],
                               'IL.10':missing_sc[:,2],
                               'IL.6':missing_sc[:,3],
                               'IL.8':missing_sc[:,4],
                               'MCP.1':missing_sc[:,5],
                               'MDC':missing_sc[:,6],
                               'MIP.1a':missing_sc[:,7],
                               'TNF.a':missing_sc[:,8],
                               'C.peptide':missing_sc[:,9],
                               'G.CSF':missing_sc[:,10],
                               'IL.22':missing_sc[:,11],
                               'Insulin':missing_sc[:,12],
                               'Leptin':missing_sc[:,13],
                               'MIP.3a':missing_sc[:,14],
                               'GRO.a':missing_sc[:,15],
                               'HGF':missing_sc[:,16],
                               'MMP.2':missing_sc[:,17],
                               'Adiponectin':missing_sc[:,18],
                               'CRP':missing_sc[:,19],
                               'GDF.15':missing_sc[:,20],
                               'TIMP.1':missing_sc[:,21],
                               'TGF.B2':missing_sc[:,22],
                               'TGF.B1':missing_sc[:,23],
                               'PPAR.y':missing_sc[:,24],
                               'HIF.1a':missing_sc[:,25],
                               'Laminin':missing_sc[:,26],
                               'HbA1c':missing_sc[:,27],
                               'CA19.9':missing_sc[:,28],
                               'Glucose':missing_sc[:,29],
                               'HDL':missing_sc[:,30],
                               'CCK':missing_sc[:,31],
                               'LDL':missing_sc[:,32],
                               'Triglyceride':missing_sc[:,33],
                               'Albumin':missing_sc[:,34],
                               'Lumican':missing_sc[:,35],
                               'ZAG':missing_sc[:,36],
                               })
    
    # Extract cat columns from missing
    ext_col1 = missing_cp['CACHEXSTAGE0VIG']
    

    # Add the extracted cat columns to missing_df
    missing_df = pd.concat([missing_df, ext_col1.rename('CACHEXSTAGE0VIG')], axis=1)
    

    missing_df.to_csv('missing_test_NCa_Ca.csv')
    data_0 = pd.read_csv('missing_test_NCa_Ca.csv')
    data_0.columns.str.strip()

    categorical = ['CACHEXSTAGE0VIG']
    
    data_cat, cat_cols_list = md.cat_conv(data_0[categorical])

    data_0.drop(categorical, axis = 1, inplace = True)
    constructor_list = [data_0]
    constructor_list.append(data_cat)
    data_in = pd.concat(constructor_list, axis=1)

    na_loc = data_in.isnull()
    data_in[na_loc] = np.nan

    # imputer = md.Midas(layer_structure = [128,128], vae_layer = True, seed = 89, input_drop = 0.75, learn_rate = 0.0001)
    imputer = md.Midas(layer_structure = [128,128], vae_layer = False, seed = 42, input_drop = 0.75, learn_rate = 0.001)
    imputer.build_model(data_in, softmax_columns = cat_cols_list)
    imputer.train_model(training_epochs = n_epoch)

    # do not update M
    imputations = imputer.generate_samples(m=20).output_list

    flat_cats = [cat for variable in cat_cols_list for cat in variable]
    for i in range(len(imputations)):
        tmp_cat = [imputations[i][x].idxmax(axis=1) for x in cat_cols_list]
        cat_df = pd.DataFrame({categorical[i]:tmp_cat[i] for i in range(len(categorical))})
        imputations[i] = pd.concat([imputations[i], cat_df], axis = 1).drop(flat_cats, axis = 1)

    mapping_ca = {'CACHEXSTAGE0VIG_0.0': 0,'CACHEXSTAGE0VIG_1.0': 1}

    imputations[0] = imputations[0].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[1] = imputations[1].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[2] = imputations[2].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[3] = imputations[3].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[4] = imputations[4].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[5] = imputations[5].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[6] = imputations[6].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[7] = imputations[7].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[8] = imputations[8].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[9] = imputations[9].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[10] = imputations[10].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[11] = imputations[11].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[12] = imputations[12].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[13] = imputations[13].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[14] = imputations[14].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[15] = imputations[15].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[16] = imputations[16].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[17] = imputations[17].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[18] = imputations[18].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[19] = imputations[19].replace({'CACHEXSTAGE0VIG': mapping_ca})

    # rescale imputations
    imputations[0][col_to_scale] = scaler.inverse_transform(imputations[0][col_to_scale])
    imputations[1][col_to_scale] = scaler.inverse_transform(imputations[1][col_to_scale])
    imputations[2][col_to_scale] = scaler.inverse_transform(imputations[2][col_to_scale])
    imputations[3][col_to_scale] = scaler.inverse_transform(imputations[3][col_to_scale])
    imputations[4][col_to_scale] = scaler.inverse_transform(imputations[4][col_to_scale])
    imputations[5][col_to_scale] = scaler.inverse_transform(imputations[5][col_to_scale])
    imputations[6][col_to_scale] = scaler.inverse_transform(imputations[6][col_to_scale])
    imputations[7][col_to_scale] = scaler.inverse_transform(imputations[7][col_to_scale])
    imputations[8][col_to_scale] = scaler.inverse_transform(imputations[8][col_to_scale])
    imputations[9][col_to_scale] = scaler.inverse_transform(imputations[9][col_to_scale])
    imputations[10][col_to_scale] = scaler.inverse_transform(imputations[10][col_to_scale])
    imputations[11][col_to_scale] = scaler.inverse_transform(imputations[11][col_to_scale])
    imputations[12][col_to_scale] = scaler.inverse_transform(imputations[12][col_to_scale])
    imputations[13][col_to_scale] = scaler.inverse_transform(imputations[13][col_to_scale])
    imputations[14][col_to_scale] = scaler.inverse_transform(imputations[14][col_to_scale])
    imputations[15][col_to_scale] = scaler.inverse_transform(imputations[15][col_to_scale])
    imputations[16][col_to_scale] = scaler.inverse_transform(imputations[16][col_to_scale])
    imputations[17][col_to_scale] = scaler.inverse_transform(imputations[17][col_to_scale])
    imputations[18][col_to_scale] = scaler.inverse_transform(imputations[18][col_to_scale])
    imputations[19][col_to_scale] = scaler.inverse_transform(imputations[19][col_to_scale])

    n=0
    for i in imputations:
        file_out_csv = "midas_test_NCa_Ca/midas_NCa_Ca_" + str(n) + ".csv"
        i.to_csv(file_out_csv, index=False)
        n += 1

    df0 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_0.csv'); df0 = df0.iloc[:,1:]
    df1 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_1.csv'); df1 = df1.iloc[:,1:]
    df2 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_2.csv'); df2 = df2.iloc[:,1:]
    df3 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_3.csv'); df3 = df3.iloc[:,1:]
    df4 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_4.csv'); df4 = df4.iloc[:,1:]
    df5 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_5.csv'); df5 = df5.iloc[:,1:]
    df6 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_6.csv'); df6 = df6.iloc[:,1:]
    df7 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_7.csv'); df7 = df7.iloc[:,1:]
    df8 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_8.csv'); df8 = df8.iloc[:,1:]
    df9 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_9.csv'); df9 = df9.iloc[:,1:]
    df10 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_10.csv'); df10 = df10.iloc[:,1:]
    df11 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_11.csv'); df11 = df11.iloc[:,1:]
    df12 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_12.csv'); df12 = df12.iloc[:,1:]
    df13 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_13.csv'); df13 = df13.iloc[:,1:]
    df14 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_14.csv'); df14 = df14.iloc[:,1:]
    df15 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_15.csv'); df15 = df15.iloc[:,1:]
    df16 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_16.csv'); df16 = df16.iloc[:,1:]
    df17 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_17.csv'); df17 = df17.iloc[:,1:]
    df18 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_18.csv'); df18 = df18.iloc[:,1:]
    df19 = pd.read_csv('midas_test_NCa_Ca/midas_NCa_Ca_19.csv'); df19 = df19.iloc[:,1:]

    # Initialize the output dataframe with the same dimensions
    output_df = pd.DataFrame(index=df0.index, columns=df0.columns)

    # Process continuous columns
    for col in df1.columns[:35]:
        output_df[col] = (abs(df0[col]) + abs(df1[col]) + abs(df2[col]) + abs(df3[col]) + abs(df4[col]) + 
                          abs(df5[col]) + abs(df6[col]) + abs(df7[col]) + abs(df8[col]) + abs(df9[col]) + 
                         abs(df10[col]) + abs(df11[col]) + abs(df12[col]) + abs(df13[col]) + abs(df14[col]) + 
                          abs(df15[col]) + abs(df16[col]) + abs(df17[col]) + abs(df18[col]) + abs(df19[col])) / 20

    # Process categorical columns
    for col in df1.columns[35:]:
        output_df[col] = pd.concat([df0[col],df1[col],df2[col],df3[col],df4[col],df5[col],df6[col],df7[col],df8[col],df9[col],
                                   df10[col],df11[col],df12[col],df13[col],df14[col],df15[col],df16[col],df17[col],df18[col],df19[col]], axis=1).median(axis=1)
        
        # median = stats.median([df0[col],df1[col],df2[col],df3[col],df4[col],df5[col],df6[col],df7[col],df8[col],df9[col],
        #                   df10[col],df11[col],df12[col],df13[col],df14[col],df15[col],df16[col],df17[col],df18[col],df19[col]], axis=0).median[0]
        # output_df[col] = median

    # Write the output dataframe to a CSV file
    output_df.to_csv('output_test_NCa_Ca.csv', index=False)

    # re-order cols in output_df
    output_df_re = output_df[['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 
                              'Leptin', 'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 
                              'Laminin', 'HbA1c', 'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG', 'CACHEXSTAGE0VIG']].copy()

    return output_df_re





In [8]:
# calculate proportion of missingness in ndata_noR

# total num of NaN in the ndata_noR
total_nan_count = X_test.isna().sum().sum()

# total num of cells in ndata_noR
total_cells = X_test.size

# proportion of NaN
NaN_proportion = total_nan_count / total_cells

print('NaN_proportions:',NaN_proportion)


NaN_proportions: 0.018243243243243244


In [9]:
# 1.35% missingness

Xy_NCa_Ca_md = midas_impl(Xy_NCa_Ca, col_to_scale, 200)

Size index: [38, 2]

Computation graph constructed

Metal device set to: Apple M1 Pro
Model initialised

Epoch: 0 , loss: 69.88936614990234
Epoch: 1 , loss: 93.7535171508789
Epoch: 2 , loss: 79.60838317871094
Epoch: 3 , loss: 82.11281204223633
Epoch: 4 , loss: 76.79610443115234
Epoch: 5 , loss: 73.67647171020508
Epoch: 6 , loss: 80.25099182128906
Epoch: 7 , loss: 76.57823181152344
Epoch: 8 , loss: 81.2846908569336
Epoch: 9 , loss: 82.81485366821289
Epoch: 10 , loss: 83.35577774047852
Epoch: 11 , loss: 89.54759979248047
Epoch: 12 , loss: 89.44928359985352
Epoch: 13 , loss: 95.17147445678711
Epoch: 14 , loss: 240.17947387695312
Epoch: 15 , loss: 134.01087188720703
Epoch: 16 , loss: 115.21092987060547
Epoch: 17 , loss: 112.03958892822266
Epoch: 18 , loss: 133.25082397460938
Epoch: 19 , loss: 72.29893493652344
Epoch: 20 , loss: 78.5397834777832
Epoch: 21 , loss: 77.67149353027344
Epoch: 22 , loss: 74.91217803955078
Epoch: 23 , loss: 84.85457611083984
Epoch: 24 , loss: 76.02645111083984
Epo

In [10]:
Xy_NCa_Ca_md

,ENA.78,IFN.y,IL.10,IL.6,IL.8,MCP.1,MDC,MIP.1a,TNF.a,C.peptide,G.CSF,IL.22,Insulin,Leptin,MIP.3a,GRO.a,HGF,MMP.2,Adiponectin,CRP,GDF.15,TIMP.1,TGF.B2,TGF.B1,PPAR.y,HIF.1a,Laminin,HbA1c,CA19.9,Glucose,HDL,CCK,LDL,Triglyceride,Albumin,Lumican,ZAG,CACHEXSTAGE0VIG
0,0.924379,0.615043,0.233952,0.457540,0.779225,1.028411,0.862336,0.560088,0.720300,0.673062,0.577212,0.400506,0.598856,0.721074,0.782062,0.873600,0.836930,0.924044,0.904242,0.830671,0.776836,0.890493,0.812666,0.946732,0.258389,0.904495,0.873085,0.833932,0.806769,0.805669,0.944378,0.908219,0.792221,0.875578,0.975151,0.928630,0.927397,1.0
1,0.800712,0.797026,0.884081,0.625474,0.784953,0.947583,0.864152,0.585399,0.794567,0.847565,0.504439,0.569218,0.664131,0.700906,0.584558,0.872167,0.711980,0.999292,0.974544,0.860851,0.868727,0.890191,0.554812,0.899712,0.025565,0.867039,0.823002,0.817571,0.830761,0.788635,0.842544,0.809578,0.806953,0.708505,0.981896,0.946960,0.945428,1.0
2,0.857912,0.732489,0.300804,0.271791,0.635667,0.899978,0.773156,0.499552,0.632092,0.793367,0.606510,0.639191,0.514993,0.708786,0.525109,0.814225,0.680589,0.962594,0.945133,0.637927,0.841551,0.890270,0.890698,0.928967,0.324017,0.736627,0.897838,0.777864,0.540681,0.869646,0.776540,0.769096,0.806148,0.815706,0.995995,0.895236,0.954264,1.0
3,0.996991,0.745828,0.296399,0.441947,0.696201,0.839570,0.845712,0.611288,0.646714,0.834878,0.844415,0.444659,0.486582,0.815702,0.370807,0.896248,0.710202,0.942915,0.945033,0.736759,0.698787,0.857468,0.943102,0.877733,0.522777,0.674962,0.904749,0.992872,0.561836,0.747352,0.745548,0.818702,0.908886,0.638269,0.973012,0.886612,0.922991,1.0
4,0.804534,0.698988,0.605932,0.509328,0.665638,0.931117,0.823649,0.631223,0.715243,0.754565,0.395874,0.607098,0.371510,0.234548,0.524374,0.798348,0.671527,1.002073,0.965902,0.726995,0.750021,0.883455,0.927897,0.881833,0.205667,0.762465,0.998835,0.746000,0.830111,0.787025,0.801534,0.870107,0.839529,0.534319,0.965111,1.006604,0.996523,1.0
5,0.769906,0.752626,0.420781,0.454321,0.627025,0.863719,0.844307,0.572915,0.697536,0.823700,0.573688,0.458172,0.437599,0.590670,0.448370,0.712540,0.926956,0.950914,0.902928,0.670376,0.687609,0.845068,0.985596,0.920969,0.257193,0.619798,0.890660,0.805584,0.440556,0.804318,0.801638,0.814898,0.836315,0.613673,0.974115,0.936278,0.914837,1.0
6,0.993186,0.737597,0.542324,0.413892,0.706827,0.846085,0.839128,0.542819,0.709649,0.844894,0.851306,0.398113,0.615137,0.729325,0.408217,0.960251,0.695701,0.911388,0.889173,0.731473,0.688593,0.890327,1.013006,0.979735,0.418237,0.674889,0.800740,0.793340,0.507373,0.774675,0.943904,0.824652,0.820744,0.656906,0.981293,0.971775,0.957961,1.0
7,0.875697,0.592046,0.098729,0.325514,0.597654,0.893950,0.848782,0.445912,0.489881,0.809985,0.657475,0.070998,0.502458,0.758076,0.376033,0.718510,0.727981,0.918317,0.947406,0.715571,0.791710,0.870474,0.664037,0.902961,0.326104,0.670019,0.937733,0.886362,0.754099,0.840492,0.748919,0.837040,0.899011,0.632750,0.979405,0.907387,0.930535,0.0
8,0.874645,0.664609,0.252893,0.376665,0.629318,0.924987,0.864284,0.540257,0.595660,0.780882,0.618983,0.002783,0.457301,0.745028,0.366833,0.816609,0.724503,0.957363,0.958878,0.768278,0.681330,0.860229,0.636392,0.931680,0.044860,0.477166,0.913701,0.798222,0.381360,0.806168,0.887517,0.809151,0.854636,0.624720,0.988446,0.936902,0.942629,1.0
9,0.828338,0.739422,0.631682,0.534492,0.671949,0.876073,0.874605,0.584725,0.737018,0.797617,0.616436,0.384932,0.441101,0.804656,0.383927,0.751616,0.691577,0.957347,0.949911,0.855975,0.823751,0.868644,0.837053,0.899229,0.339406,0.965165,0.879191,0.745712,0.461766,0.976367,0.878639,0.644140,0.923082,0.796664,0.971684,0.971275,0.965901,1.0


In [11]:
# save to csv file

# 
Xy_NCa_Ca_md.to_csv('data/Xy_test_NCa_Ca_md.csv', index=False)